In [1]:
import numpy as np
import pandas as pd
import os
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms, models
from torchvision.datasets import ImageFolder
from torchvision.models import resnet50

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

# Data Preprocessing

In [3]:
train_dir = '../input/face-mask-12k-images-dataset/Face Mask Dataset/Train/'
val_dir = '../input/face-mask-12k-images-dataset/Face Mask Dataset/Validation/'
test_dir = '../input/face-mask-12k-images-dataset/Face Mask Dataset/Test/'
#print(os.walk('/kaggle/input'))
#for dirname, _, filenames in os.walk('/kaggle/input'):
   # for filename in filenames:        
        #print(os.path.join(dirname, filename))



#train_list = []
#for 

In [4]:
train_transforms = transforms.Compose([
#                                        transforms.RandomRotation(30),
                                       transforms.Resize((224, 224)),
#                                        transforms.RandomHorizontalFlip(),
#                                        transforms.Grayscale(3),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([
                                      transforms.Resize((224, 224)),
#                                       transforms.Grayscale(3),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])
val_transforms = transforms.Compose([
                                      transforms.Resize((224, 224)),
#                                       transforms.Grayscale(3),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])
train_data = datasets.ImageFolder(train_dir, transform=train_transforms)
test_data = datasets.ImageFolder(test_dir, transform=test_transforms)
val_data = datasets.ImageFolder(val_dir, transform=val_transforms)
train_loader = DataLoader(dataset=train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=32)
val_loader = DataLoader(dataset=val_data, batch_size=32)

In [5]:
model = models.resnet18(pretrained=True)
print(model)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [6]:
for param in model.parameters():
    param.requires_grad = False
model.avgpool = nn.AdaptiveAvgPool2d(1)
model.fc = nn.Sequential(nn.Flatten(), 
    nn.Linear(512, 128),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(128, 1),
    nn.Sigmoid())
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr= 0.001)

In [7]:
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

# Training a model

In [8]:
num_epochs = 5
n_total_steps = len(train_loader)
model.train()
valid_loss_min = np.Inf
for epoch in range(num_epochs):
    valid_loss = 0.0
    for i, (images, labels) in enumerate(train_loader):
        images = images.float().to(device)
        labels = torch.unsqueeze(labels, 1)
        labels = labels.float().to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print(f'Epoch [{epoch + 1} / {num_epochs}], Step [{i + 1} / {n_total_steps}], Loss: {loss.item():.4f}')
        valid_loss = loss.item()
        
      #  if valid_loss <= valid_loss_min:
      #      print(f'Validation loss decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}))
      #      valid_loss_min = valid_loss

Epoch [1 / 5], Step [100 / 313], Loss: 0.0431
Epoch [1 / 5], Step [200 / 313], Loss: 0.2271
Epoch [1 / 5], Step [300 / 313], Loss: 0.0644
Epoch [2 / 5], Step [100 / 313], Loss: 0.3012
Epoch [2 / 5], Step [200 / 313], Loss: 0.0502
Epoch [2 / 5], Step [300 / 313], Loss: 0.0020
Epoch [3 / 5], Step [100 / 313], Loss: 0.0032
Epoch [3 / 5], Step [200 / 313], Loss: 0.0223
Epoch [3 / 5], Step [300 / 313], Loss: 0.0059
Epoch [4 / 5], Step [100 / 313], Loss: 0.0071
Epoch [4 / 5], Step [200 / 313], Loss: 0.0015
Epoch [4 / 5], Step [300 / 313], Loss: 0.0007
Epoch [5 / 5], Step [100 / 313], Loss: 0.0091
Epoch [5 / 5], Step [200 / 313], Loss: 0.0020
Epoch [5 / 5], Step [300 / 313], Loss: 0.1383


# Testing a model

In [9]:
#model.load_state_dict(torch.load('model.pt'))
model.eval()
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = torch.unsqueeze(labels, 1)
        labels = labels.to(device)
        predictions = model(images)
        predictions = (predictions >= 0.5)
        
        n_samples += labels.shape[0]
        n_correct += (predictions == labels).sum().item()
#         print(n_correct, n_samples)
    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network on the test images: {acc} %')

Accuracy of the network on the test images: 99.79838709677419 %
